In [2]:
# if dash is not installed please uncomment and install dash
#!pip install dash

# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

     |████████████████████████████████| 7.3 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 288 kB 49.2 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 54.8 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3804 sha256=066d7cb130f0c81efa79268fed202f574e9e0502c9ae61a7a2ee5eae4c0ff32a
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/e4/f3/16724791571a955a46d54650510c98c04ab7d339626aee27cc
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4069 sha256=23a2aaa07b603493ead3aa5ae4275ff1c18519b42eeb5a7d3920e2fadd26fbc4
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/73/d8/8d/92f612c03c895f19bcc56a6c54be7bb41aaa698012a5624f60
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl 

<ipython-input-2-aa821cfd31b2>:7: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-aa821cfd31b2>:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [5]:
# Read the airline data into pandas dataframe
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-09-16 06:55:08--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-09-16 06:55:09 (55.1 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [10]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options = [{'label': 'All Sites', 'value': 'ALL'},
                                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}],
                                             placeholder='Select a Launch Site here',
                                             searchable=True,
                                             value='ALL',
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                marks={0:'0',
                                                      2500:'2500',
                                                      5000:'5000',
                                                      7500:'7500',
                                                      10000:'10000'},
                                                value = [min_payload,max_payload]),
                                html.Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def update_chart(site_dropdown):
    if site_dropdown == 'ALL':
        piechart = px.pie(spacex_df, values='class', names='Launch Site')
        return piechart
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site_dropdown].groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        piechart = px.pie(filtered_df, values='class count', names='class')
        return piechart

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def update_chart(slider_range,site_dropdown):
    
    filtered_df_1 = spacex_df[(slider_range[0] <= spacex_df['Payload Mass (kg)']) & (spacex_df['Payload Mass (kg)']<= slider_range[1])]
    if site_dropdown == 'ALL':
        scatter = px.scatter(filtered_df_1, x='Payload Mass (kg)', y='class', color="Booster Version Category")
        return scatter
         
    else:
        filtered_df_2= filtered_df_1[filtered_df_1['Launch Site'] == site_dropdown]
        scatter = px.scatter(filtered_df_2, x='Payload Mass (kg)', y='class', color="Booster Version Category")
        return scatter


# Run the app
if __name__ == '__main__':
    app.run_server()